In [1]:
import os
os.chdir('D:\Machine Learning\Paper\Object Detection\ssd.pytorch-master')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import argparse
from torch.autograd import Variable
import torch.utils.data as data

import sys
sys.path.append('D:\Machine Learning\Paper\Object Detection\ssd.pytorch-master')
from easydict import EasyDict

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.init as init
import argparse
from torch.autograd import Variable
import torch.utils.data as data
from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time

In [3]:
# -*- coding: utf-8 -*-
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from data import v2 as cfg

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from layers import *
from data import v2
import os

In [5]:
from data import v2, v1, AnnotationTransform, VOCDetection, detection_collate, VOCroot, VOC_CLASSES
from utils.augmentations import SSDAugmentation
from layers.modules import MultiBoxLoss
from ssd import build_ssd
import numpy as np
import time

def str2bool(v):
    return v.lower() in ("yes", "true", "t", "1")

config = EasyDict()
config.version='v2'
config.basenet='vgg16_reducedfc.pth'
config.jaccard_threshold=0.5
config.batch_size=8
config.resume=None
config.num_workers=4
config.iterations=120000
config.start_iter=0
config.cuda=False
config.lr=1e-3
config.momentum=0.9
config.weight_decay=5e-4
config.gamma=0.1
config.log_iters=True
config.visdom=False
config.send_images_to_visdom=False
config.save_folder='weights/'
config.voc_root=VOCroot

In [6]:
if config.cuda and torch.cuda.is_available():
    torch.set_default_tensor_type('torch.cuda.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')

cfg = (v1, v2)[config.version == 'v2']

if not os.path.exists(config.save_folder):
    os.mkdir(config.save_folder)

train_sets = [('2007', 'trainval'), ('2012', 'trainval')]
# train_sets = 'train'
ssd_dim = 300  # only support 300 now
means = (104, 117, 123)  # only support voc now
num_classes = len(VOC_CLASSES) + 1
batch_size = config.batch_size
accum_batch_size = 32
iter_size = accum_batch_size / batch_size
max_iter = 120000
weight_decay = 0.0005
stepvalues = (80000, 100000, 120000)
gamma = 0.1
momentum = 0.9
if config.visdom:
    import visdom
    viz = visdom.Visdom()

ssd_net = build_ssd('train', 300, num_classes)
net = ssd_net

if config.resume:
    print('Resuming training, loading {}...'.format(config.resume))
    ssd_net.load_weights(config.resume)
else:
    vgg_weights = torch.load(config.save_folder + config.basenet)
    print('Loading base network...')
    ssd_net.vgg.load_state_dict(vgg_weights)

if config.cuda:
    net = net.cuda()


def xavier(param):
    init.xavier_uniform(param)


def weights_init(m):
    if isinstance(m, nn.Conv2d):
        xavier(m.weight.data)
        m.bias.data.zero_()

if not config.resume:
    print('Initializing weights...')
    # initialize newly added layers' weights with xavier method
    ssd_net.extras.apply(weights_init)
    ssd_net.loc.apply(weights_init)
    ssd_net.conf.apply(weights_init)

optimizer = optim.SGD(net.parameters(),lr=config.lr,momentum=config.momentum,weight_decay=config.weight_decay)
# criterion = MultiBoxLoss(num_classes, 0.5, True, 0, True, 3, 0.5, False, args.cuda)

Loading base network...
Initializing weights...


In [8]:
def point_form(boxes):
    """ Convert prior_boxes to (xmin, ymin, xmax, ymax)
    representation for comparison to point form ground truth data.
    Args:
        boxes: (tensor) center-size default boxes from priorbox layers.
    Return:
        boxes: (tensor) Converted xmin, ymin, xmax, ymax form of boxes.
    """
    return torch.cat((boxes[:, :2] - boxes[:, 2:]/2,     # xmin, ymin
                     boxes[:, :2] + boxes[:, 2:]/2), 1)  # xmax, ymax

In [9]:
def intersect(box_a, box_b):
    """ We resize both tensors to [A,B,2] without new malloc:
    [A,2] -> [A,1,2] -> [A,B,2]
    [B,2] -> [1,B,2] -> [A,B,2]
    Then we compute the area of intersect between box_a and box_b.
    Args:
      box_a: (tensor) bounding boxes, Shape: [A,4].
      box_b: (tensor) bounding boxes, Shape: [B,4].
    Return:
      (tensor) intersection area, Shape: [A,B].
    """
    A = box_a.size(0)
    B = box_b.size(0)
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]

In [10]:
def encode(matched, priors, variances):
    """Encode the variances from the priorbox layers into the ground truth boxes
    we have matched (based on jaccard overlap) with the prior boxes.
    Args:
        matched: (tensor) Coords of ground truth for each prior in point-form
            Shape: [num_priors, 4].
        priors: (tensor) Prior boxes in center-offset form
            Shape: [num_priors,4].
        variances: (list[float]) Variances of priorboxes
    Return:
        encoded boxes (tensor), Shape: [num_priors, 4]
    """

    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = torch.log(g_wh) / variances[1]
    # return target for smooth_l1_loss
    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,4]

In [11]:
def jaccard(box_a, box_b):
    """Compute the jaccard overlap of two sets of boxes.  The jaccard overlap
    is simply the intersection over union of two boxes.  Here we operate on
    ground truth boxes and default boxes.
    E.g.:
        A ∩ B / A ∪ B = A ∩ B / (area(A) + area(B) - A ∩ B)
    Args:
        box_a: (tensor) Ground truth bounding boxes, Shape: [num_objects,4]
        box_b: (tensor) Prior boxes from priorbox layers, Shape: [num_priors,4]
    Return:
        jaccard overlap: (tensor) Shape: [box_a.size(0), box_b.size(0)]
    """
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) * (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) * (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]

In [12]:
def log_sum_exp(x):
    """Utility function for computing log_sum_exp while determining
    This will be used to determine unaveraged confidence loss across
    all examples in a batch.
    Args:
        x (Variable(tensor)): conf_preds from conf layers
    """
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max

In [13]:
def match(threshold, truths, priors, variances, labels, loc_t, conf_t, idx):
    """Match each prior box with the ground truth box of the highest jaccard
    overlap, encode the bounding boxes, then return the matched indices
    corresponding to both confidence and location preds.
    Args:
        threshold: (float) The overlap threshold used when mathing boxes.
        truths: (tensor) Ground truth boxes, Shape: [num_obj, num_priors].
        priors: (tensor) Prior boxes from priorbox layers, Shape: [n_priors,4].
        variances: (tensor) Variances corresponding to each prior coord,
            Shape: [num_priors, 4].
        labels: (tensor) All the class labels for the image, Shape: [num_obj].
        loc_t: (tensor) Tensor to be filled w/ endcoded location targets.
        conf_t: (tensor) Tensor to be filled w/ matched indices for conf preds.
        idx: (int) current batch index
    Return:
        The matched indices corresponding to 1)location and 2)confidence preds.
    """
    # jaccard index
    overlaps = jaccard(truths,point_form(priors))
    # (Bipartite Matching)
    # [1,num_objects] best prior for each ground truth
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True)
    # [1,num_priors] best ground truth for each prior
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True)
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    best_truth_overlap.index_fill_(0, best_prior_idx, 2)  # ensure best prior
    # TODO refactor: index  best_prior_idx with long tensor
    # ensure every gt matches with its prior of max overlap
    for j in range(best_prior_idx.size(0)):
        best_truth_idx[best_prior_idx[j]] = j
    matches = truths[best_truth_idx]          # Shape: [num_priors,4]
    conf = labels[best_truth_idx] + 1         # Shape: [num_priors]
    conf[best_truth_overlap < threshold] = 0  # label as background
    loc = encode(matches, priors, variances)
    loc_t[idx] = loc    # [num_priors,4] encoded offsets to learn
    conf_t[idx] = conf  # [num_priors] top class label for each prior

In [14]:
net.train()
dataset = VOCDetection(config.voc_root, train_sets, SSDAugmentation(ssd_dim, means), AnnotationTransform())
data_loader = data.DataLoader(dataset, batch_size,shuffle=True, collate_fn=detection_collate, pin_memory=True)
batch_iterator = iter(data_loader)

In [15]:
images, targets = next(batch_iterator)

if config.cuda:
    images = Variable(images.cuda())
    targets = [Variable(anno.cuda(), volatile=True) for anno in targets]
else:
    images = Variable(images)
    targets = [Variable(anno, volatile=True) for anno in targets]

In [16]:
out = net(images)

In [17]:
targets[0]

Variable containing:
  0.1823   0.4275   0.6569   0.8442   6.0000
  0.3420   0.4384   0.4122   0.5435  14.0000
  0.2616   0.4656   0.2978   0.5543  14.0000
[torch.FloatTensor of size 3x5]

In [16]:
use_gpu = False
num_classes = num_classes
threshold = 0.5
background_label = 0
encode_target = False
use_prior_for_matching = True
do_neg_mining = True
negpos_ratio = 3
neg_overlap = 0.5
variance = cfg['variance']

In [17]:
predictions = out

In [18]:
loc_data, conf_data, priors = predictions
num = loc_data.size(0)
priors = priors[:loc_data.size(1), :]
num_priors = (priors.size(0))
num_classes = num_classes

# match priors (default boxes) and ground truth boxes
loc_t = torch.Tensor(num, num_priors, 4)
conf_t = torch.LongTensor(num, num_priors)
for idx in range(num):
    truths = targets[idx][:, :-1].data
    labels = targets[idx][:, -1].data
    defaults = priors.data
    match(threshold, truths, defaults, variance, labels,
  loc_t, conf_t, idx)
if use_gpu:
    loc_t = loc_t.cuda()
    conf_t = conf_t.cuda()
# wrap targets
loc_t = Variable(loc_t, requires_grad=False)
conf_t = Variable(conf_t, requires_grad=False)

pos = conf_t > 0
num_pos = pos.sum(dim=1, keepdim=True)

# Localization Loss (Smooth L1)
# Shape: [batch,num_priors,4]
pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
loc_p = loc_data[pos_idx].view(-1, 4)
loc_t = loc_t[pos_idx].view(-1, 4)
loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

# Compute max conf across batch for hard negative mining
batch_conf = conf_data.view(-1, num_classes)

In [90]:
conf_t.shape

torch.Size([8, 8732])

In [19]:
loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

In [20]:
test = torch.nn.functional.log_softmax(batch_conf,dim=1).gather(1,conf_t.view(-1,1))

In [21]:
pos.shape

torch.Size([8, 8732])

In [22]:
loss_c.shape

torch.Size([69856, 1])

In [27]:
d = Variable(torch.FloatTensor(loss_c.data.numpy()))

In [28]:
d.shape

torch.Size([69856, 1])

In [30]:
# Hard Negative Mining
loss_c[pos.view(-1,1)] = 0  # filter out pos boxes for now

In [32]:
loss_c = loss_c.view(num, -1)

In [33]:
loss_c.shape

torch.Size([8, 8732])

In [34]:
_, loss_idx = loss_c.sort(1, descending=True)

In [36]:
loss_idx

Variable containing:
 4705  2328  5228  ...   6983  8708  8731
  334  3816  3976  ...   8728  8729  8730
 1588  1568   156  ...   3855  3551  3851
       ...          ⋱          ...       
 2612  2616  1878  ...   8192  8196  8202
 1589   905  4245  ...   8215  8213  6863
 1273  1429   472  ...   6376  6377  6383
[torch.LongTensor of size 8x8732]

In [51]:
loss_idx[0,8731]

Variable containing:
 8731
[torch.LongTensor of size 1]

In [43]:
idx_rank

Variable containing:
 3197  4545  2254  ...   8708  8709  8731
 4382  4475  2049  ...   8730  8731  5653
 2738  3081  3467  ...   5716  5597  5709
       ...          ⋱          ...       
 5780  8112  2219  ...   5636  5547  5626
 5110  7027  4483  ...   5893  5822  5870
 5156  6987  2205  ...   5005  4946  4994
[torch.LongTensor of size 8x8732]

In [45]:
a = torch.rand([3,3])

In [46]:
a


 0.0093  0.1851  0.4908
 0.0170  0.6716  0.3332
 0.6040  0.2060  0.3216
[torch.FloatTensor of size 3x3]

In [47]:
a.sort(1)

(
  0.0093  0.1851  0.4908
  0.0170  0.3332  0.6716
  0.2060  0.3216  0.6040
 [torch.FloatTensor of size 3x3], 
  0  1  2
  0  2  1
  1  2  0
 [torch.LongTensor of size 3x3])

In [42]:
_, idx_rank = loss_idx.sort(1)

In [52]:
pos

Variable containing:
    0     0     0  ...      1     1     1
    0     0     0  ...      1     1     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.ByteTensor of size 8x8732]

In [53]:
num_pos = pos.long().sum(1, keepdim=True)

In [54]:
num_pos

Variable containing:
   25
    6
    5
   24
   10
    7
   16
   13
[torch.LongTensor of size 8x1]

In [56]:
pos.size(1)

8732

In [57]:
num_neg = torch.clamp(negpos_ratio*num_pos, max=pos.size(1)-1)

In [58]:
num_neg

Variable containing:
   75
   18
   15
   72
   30
   21
   48
   39
[torch.LongTensor of size 8x1]

In [59]:
idx_rank

Variable containing:
 3197  4545  2254  ...   8708  8709  8731
 4382  4475  2049  ...   8730  8731  5653
 2738  3081  3467  ...   5716  5597  5709
       ...          ⋱          ...       
 5780  8112  2219  ...   5636  5547  5626
 5110  7027  4483  ...   5893  5822  5870
 5156  6987  2205  ...   5005  4946  4994
[torch.LongTensor of size 8x8732]

In [61]:
neg = idx_rank < num_neg.expand_as(idx_rank)

In [60]:
num_neg.expand_as(idx_rank)

Variable containing:
   75    75    75  ...     75    75    75
   18    18    18  ...     18    18    18
   15    15    15  ...     15    15    15
       ...          ⋱          ...       
   21    21    21  ...     21    21    21
   48    48    48  ...     48    48    48
   39    39    39  ...     39    39    39
[torch.LongTensor of size 8x8732]

In [62]:
neg

Variable containing:
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.ByteTensor of size 8x8732]

In [87]:
conf.shape

NameError: name 'conf' is not defined

In [86]:
conf_t[0]

Variable containing:
  0
  0
  0
 ⋮ 
 18
 15
 18
[torch.LongTensor of size 8732]

In [63]:
conf_data.shape

torch.Size([8, 8732, 21])

In [79]:
pos.shape

torch.Size([8, 8732])

In [64]:
# Confidence Loss Including Positive and Negative Examples
pos_idx = pos.unsqueeze(2).expand_as(conf_data)
neg_idx = neg.unsqueeze(2).expand_as(conf_data)

In [68]:
(pos_idx+neg_idx).gt(0).shape

torch.Size([8, 8732, 21])

In [70]:
pos_idx.shape

torch.Size([8, 8732, 21])

In [71]:
neg_idx.shape

torch.Size([8, 8732, 21])

In [78]:
(pos_idx+neg_idx).gt(0)

Variable containing:
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
       ...          ⋱          ...       
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
    0     0     0  ...      0     0     0
[torch.ByteTensor of size 220x21]

In [80]:
conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, num_classes)

In [82]:
conf_p.shape

torch.Size([424, 21])

In [ ]:
targets_weighted = conf_t[(pos+neg).gt(0)]
loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

# Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N

N = num_pos.data.sum()
loss_l /= N
loss_c /= N

In [ ]:
class MultiBoxLoss(nn.Module):
    """SSD Weighted Loss Function
    Compute Targets:
        1) Produce Confidence Target Indices by matching  ground truth boxes
           with (default) 'priorboxes' that have jaccard index > threshold parameter
           (default threshold: 0.5).
        2) Produce localization target by 'encoding' variance into offsets of ground
           truth boxes and their matched  'priorboxes'.
        3) Hard negative mining to filter the excessive number of negative examples
           that comes with using a large number of default bounding boxes.
           (default negative:positive ratio 3:1)
    Objective Loss:
        L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        Where, Lconf is the CrossEntropy Loss and Lloc is the SmoothL1 Loss
        weighted by α which is set to 1 by cross val.
        Args:
            c: class confidences,
            l: predicted boxes,
            g: ground truth boxes
            N: number of matched default boxes
        See: https://arxiv.org/pdf/1512.02325.pdf for more details.
    """

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        self.num_classes = num_classes
        self.threshold = overlap_thresh
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        self.do_neg_mining = neg_mining
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']

    def forward(self, predictions, targets):
        """Multibox Loss
        Args:
            predictions (tuple): A tuple containing loc preds, conf preds,
            and prior boxes from SSD net.
                conf shape: torch.size(batch_size,num_priors,num_classes)
                loc shape: torch.size(batch_size,num_priors,4)
                priors shape: torch.size(num_priors,4)

            ground_truth (tensor): Ground truth boxes and labels for a batch,
                shape: [batch_size,num_objs,5] (last idx is the label).
        """
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        # wrap targets
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)

        # Localization Loss (Smooth L1)
        # Shape: [batch,num_priors,4]
        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)

        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c[pos] = 0  # filter out pos boxes for now
        loss_c = loss_c.view(num, -1)
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N

        N = num_pos.data.sum()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c

In [ ]:
net.train()
# loss counters
loc_loss = 0  # epoch
conf_loss = 0
epoch = 0
print('Loading Dataset...')

dataset = VOCDetection(config.voc_root, train_sets, SSDAugmentation(ssd_dim, means), AnnotationTransform())

epoch_size = len(dataset) // config.batch_size
print('Training SSD on', dataset.name)
step_index = 0

In [ ]:
batch_iterator = None
data_loader = data.DataLoader(dataset, batch_size, shuffle=True, collate_fn=detection_collate,pin_memory=True)

In [ ]:
def adjust_learning_rate(optimizer, gamma, step):
    """Sets the learning rate to the initial LR decayed by 10 at every specified step
    # Adapted from PyTorch Imagenet example:
    # https://github.com/pytorch/examples/blob/master/imagenet/main.py
    """
    lr = config.lr * (gamma ** (step))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr